# 0.0 Imports

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import inflection 

from matplotlib import pyplot as plt
from IPython.display import HTML

## 0.1 Helper Functions

### Set Display

In [2]:
def notebook_settings():

    plt.style.use( 'ggplot' )
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24

    display(HTML('<style>.container {width:100% !important;}</style>'))
    pd.options.display.max_columns=None
    pd.options.display.max_rows=None
    pd.set_option( 'display.expand_frame_repr', False)

    sns.set()
notebook_settings()

## 0.2 Load Data

In [4]:
data = pd.read_csv('../data/raw/data.csv', encoding='unicode_escape')

# Planification (IoT)

In [ ]:
# Input
- Business problem: select the most value clients to integrate the loyalty program
- Data set

# Output
- The indication of the people who integrate the Insiders group
- Report with the answers to the business questions

# Tasks
**Perguntas:**

1. **Quem são as pessoas elegíveis para participar do programa Insiders?**
    1. O que é ser elegivel?
    2. O que são clientes de maior valor?
    3. Faturamento
        1. Alto ticket médio
        2. Alto LTV
        3. Baixa recência (recorrência alta e recente)
        4. Alto basket size (tamanho da cesta de compra
        5. Baixa probabilidade de Churn
        6. Alta Previsão de LTV
        7. Alta propensão de compra
    4. Custo
        1. Baixo taxa de devolução
        2. 
    5. Experiência de compra
        1. Média alta das avaliações

2. **Quantos clientes farão parte do grupo?**
    1. Número total de clientes
    2. % do grupo Insiders

3. **Quais as principais características destes clientes?**
    1. **Características do cliente**
        1. Cliente
            1. Idade
            2. Localização
        2. Consumo
            1. Atributos da clusterização
    
4. **Qual a porcentagem de contribuição do faturamento, vinda dos Insiders?**
    1. Faturamento total ano
    2. Faturamento grupo Insiders

5. **Qual a expectativa de faturamento desse grupo para os próximos meses?**
    1. LTV grupo Insiders
    2. Análise de Cohort

6. **Quais as condições para uma pessoa participar do grupo?**
    1. Definir periodicidade (1 mes, 3 meses )
    2. A pessoa precisa ser similar com uma pessoa do grupo

7. **Quais as condições para uma pessoas ser removida do grupo?**
    1. Definir periodicidade
    2. A pessoa precisa ser disimilar ao grupo

8. **Qual a garantia que o programa é melhor que o restante da base?**
    1. Test A/B
    2. Teste de hipótese

9. **Quais ações o time de marketing pode realizar para aumentar o faturamento?**
    1. Desconto
    2. Preferência de compra 
    3. frete
    4. visita a empresa

# Process
1. Data exploration
    1. Statistical descriptio
    2. Count the classes

2. Dat cleaning
    1. Check and fillout Nan
    2. Look for outliers and handle them

3. EDA
    1. Explore the distribuition of the numerical features (Unvariate Analysis)
    2. Explore the correlation with the value spended in the shop
    3. Answer the business question
    
4. Feature Engineering
    1. RFM segmentation
    2. Create RMF Score 


# 1.0 Data Description

### 1.1 Data Dimensions

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [13]:
data.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### 1.2 Rename Columns

In [10]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore( x ) # change to snakecase
cols_new = list( map( snakecase, cols_old ))

data.columns = cols_new # define the new columns

### 1.3 Check and fillout Nan

In [12]:
# check Na
data.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

In [18]:
# drop Na
data = data.dropna()

### 1.4 Change dtypes

In [19]:
# invoice data to datetime
data['invoice_date'] = pd.to_datetime(data['invoice_date'])

### 1.5 Descriptive Statistics

#### Numerical

In [16]:
data.describe().T

,count,mean,min,25%,50%,75%,max,std
quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
invoice_date,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
unit_price,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
customer_id,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


#### Categorical

In [17]:
data.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

# 2.0 Feature Engineering

In [80]:
df1 = data.copy()

In [81]:
df_ref = df1['customer_id'].drop_duplicates( ignore_index=True)

In [82]:
# revenue
df1['revenue'] = df1['unit_price'] * df1['quantity']

# Monetary
ref = df1[['customer_id', 'revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, ref, on='customer_id', how='left')

# recency
recency = df1[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
recency['recency_days'] = (df1['invoice_date'].max() -  recency['invoice_date']).dt.days
recency = recency[['customer_id', 'recency_days']]
df_ref = pd.merge(df_ref, recency, on='customer_id', how='left')

# frequency
frequency = df1[['invoice_no', 'customer_id']].drop_duplicates().groupby('customer_id').count().reset_index()
frequency.columns = ['customer_id', 'frequency']
df_ref = pd.merge(df_ref, frequency, on='customer_id', how='left')

# 3.0 Data Filtering

# 4.0 EDA

## 4.1 Hypothesis Map

## 4.2 Univariate Analysis

# 5.0 Data Preparation

### Standardization, Rescaling and Encoding

# 6.0 Feature Selection

## 6.1 Split Train & Validation

## 6.2 Fature Selection

### Boruta

### Fature Importance

# 7.0 Hyperparameter Fine-Tunning

In [ ]:
df2 

# 8.0 Machine Learnign Modeling

# 9.0 Cluster Analysis

# 10.0 Deploy